In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import imageio
import scipy
from sklearn import preprocessing
from scipy import signal
import timit_utils as tu
import timit_utils.audio_utils as au
import timit_utils.drawing_utils as du
import librosa
import pywt
import random
from python_speech_features import mfcc
from python_speech_features import logfbank
from scipy.signal import butter, lfilter
import speech_recognition as sr
import re

corpus = tu.Corpus('TIMIT')
train = corpus.train
test = corpus.test
corpus.train, corpus.test


(<timit_utils.core.SubCorpus at 0x27dbd7ba898>,
 <timit_utils.core.SubCorpus at 0x27dbd7bab00>)

In [135]:
data_set = []
label_id = []
for p in range(len(train.people)):
    count = 0
    for s in range(len(train.person_by_index(p).sentences)):
        sentence = train.person_by_index(p).sentence_by_index(s)
        label_id.append(train.person_by_index(p).name)
        data_set.append(sentence)
for p in range(len(test.people)):
    count = 0
    for s in range(len(test.person_by_index(p).sentences)):
        sentence = test.person_by_index(p).sentence_by_index(s)
        label_id.append(test.person_by_index(p).name)
        data_set.append(sentence)

le1 = preprocessing.LabelEncoder()
le1.fit(label_id)
label_id = le1.transform(label_id)
N = 20 # number of speakers being considered
sub_data_set = []
speak_ind = []
S = [120,439,531,191,481,179,309,201,33,90,602,200,6,451,444,337,466,599,214,611]
for ind1 in range(N):
    ind2 = S[ind1]
    speak_ind.append(ind2)
act_ind = []
for ID in speak_ind:
    act_ind= np.where(label_id == ID)
    sub_data_set = sub_data_set+data_set[act_ind[0][0]:act_ind[0][-1]+1]

In [150]:
wrongs = []
corrects = []
for s0 in sub_data_set:
    sentence_w = s0.words_df.index.values
    sentence_a = s0.raw_audio
    sentence_a = np.int16(sentence_a*0.3*32767)
    scipy.io.wavfile.write('temp.wav', 16000, sentence_a)
    r = sr.Recognizer()
    with sr.AudioFile('temp.wav') as source:
        audio = r.record(source)  # read the entire audio file
    wordList = re.sub("[^\w]", " ",  r.recognize_google(audio)).lower().split()
    wrong = 0
    correct = 0
    for word in sentence_w:
        if word in wordList:
            correct = correct+1
        else:
            wrong = wrong+1
    wrongs = wrongs+[wrong]
    corrects = corrects+[correct]
print(wrongs)
print(corrects)


[0, 1, 0, 0, 2, 1, 2, 0, 0, 0, 0, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 1, 3, 1, 4, 3, 2, 0, 0, 3, 0, 4, 0, 2, 2, 1, 2, 0, 2, 0, 0, 1, 0, 0, 3, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 2, 2, 1, 3, 2, 0, 0, 1, 1, 2, 0, 1, 1, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 2, 0, 0, 0, 2, 1, 0, 1, 4, 0, 1, 3, 4, 2, 0, 1, 0, 1, 5, 2, 0, 2, 0, 1, 1, 3, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 2, 2, 3, 1, 1, 0, 2, 0, 1, 2, 0, 1, 2, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 3, 3, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 2, 1, 2, 2, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 2]
[11, 9, 12, 13, 5, 6, 7, 5, 9, 6, 11, 9, 9, 11, 12, 7, 8, 6, 4, 13, 11, 9, 10, 6, 2, 7, 5, 9, 11, 3, 11, 6, 5, 3, 10, 7, 5, 12, 6, 6, 11, 9, 8, 6, 12, 7, 6, 8, 10, 8, 11, 9, 10, 8, 7, 8, 11, 8, 7, 6, 10, 9, 6, 16, 5, 7, 7, 4, 7, 5, 11, 9, 9, 5, 13, 7, 5, 8, 5, 8, 11, 9, 8, 11, 7, 6, 7, 10, 6, 6, 11, 9, 14, 6, 5, 8, 7, 4, 9, 6, 11, 9, 2, 6, 13, 5, 2, 5, 10, 8, 11, 9, 7, 7, 7, 5, 5, 11, 7, 5, 11, 9

In [151]:
print(sum(wrongs))
print(sum(corrects))
print(sum(corrects)/(sum(corrects)+sum(wrongs)))

167
1586
0.9047347404449515


In [152]:
print(wrongs.count(0))
print(wrongs.count(0)/(N*10))

100
0.5
